#Localización del sonido por animales binaurales.

T. N. Parks y E. W. Rubel sugirieron un mecanismo de cómo los pollos pueden localizar sonidos empleando una red neuronal simple. La red neuronal se muestra en la siguiente figura:

![](https://raw.githubusercontent.com/Oscarhg42/Neural-Systems/Oscar/images/photo.JPG)

Esta red describe caracterísitcas escenciales del núcleo auditivo de los pollos. En este modelo cada oído tiene varios axones que salen de él, y cada uno de estos axones llega a una direfente neurona central localizada en el tálamo. Cuando una de las neuronas centrales reciben simultaneamente estimulación de ambos lados, derecho e ixquierdo, ésta dispara un potencial de acción. Si solo un lado la excita, no hay disparo. La señal producida por una de estas neuronas centrales cuasa una respuesta muscular, y el pollo responde ubicando de donde proviene el sonido girando su cabeza.



##Una red binaural

Una onda acústica entrando en un oído causa que la membrana basilar vibre en una región que corresponde a la frecuencia de la onda acústica. Como respuesta pequeñas céluas de cabello en esa región exitan un axón de una neurona causando que ésta dispare un potencial de acción. 
Este proceso se modela con un VCON. La señal causada por un estímulo en el oído derecho llega a una neurona central que reacciona contrayendo algunos músculos haciendo rotar al pollo la cabeza hacia la derecha, lo mismo sucede cuando entra un estímulo de lado derecho. el sistema de ecuaciones es el siguiente (véase la refernecia [Frank C. Hoppensteadt]):

$$\dot{x}_0=\omega _0+V(x_0) +L(t) $$

$$\ddot{x}_j+\dot{x}_j=\omega_j+V(x_j)+V(x_{j-1})$$

$$\ddot{y}_j+\dot{y}_j=\mu _j +V(y_j)+V(x_j)+V(z_j)$$

$$\dot{z}_j=\omega _j + V(z_j)+V(z_{j-1})$$

$$\dot{z_0}=\omega _0 +V(z_0)+R(t)$$

para $j=1,..,N$ . Un pulso entrando por el lado derecho seguido de un entrando por el oído izquierdo produce pulsos moviéndose a lo largo de las respectivas cadenas de axones.

Se necesita hacer un cambio de variables para pasar aun sistema de ecuaciones de primer orden:

$$\dot{x_j}=u_j \  ; \ \dot{y}=v_j $$

Con esto el sistema se transforma en:

$$\dot{x}_0=\omega _0+V(x_0) +L(t) $$

$$\dot{x_j}=u_j$$

$$\dot{u}_j=\omega_j+V(x_j)+V(x_{j-1})-\dot{x}_j$$

$$\dot{y}=v_j$$

$$\dot{v}_j=\mu _j +V(y_j)+V(x_j)+V(z_j)-\dot{y}_j$$

$$\dot{z}_j=\omega _j + V(z_j)+V(z_{j-1})$$

$$\dot{z_0}=\omega _0 +V(z_0)+R(t)$$

Sin embargo como ya se hizo notar en la parte de [modelos simples de neuronas](https://github.com/Oscarhg42/Neural-Systems/blob/Oscar/Modelos%20b%C3%A1sicos%20de%20redes..ipynb), no se lograron obtener resultados adecuados para ecuaciones difernciales de segundo orden.

Debido a la cantidad ecuaciones diferenciales que deben resolverse para la red de la figura anterior, solo trabajaremos con 9 neuronas.

Debido a que el tálamo es el encargado de procesar los estímulos sensoriales en el cerebro una buena idea para modelar esta red neuronal es partir de las ecuaciones  del modelo de atoll o modelo de [circuito talamocortical](PONER LINK!!!!!) pero haciendo unas pequeñas modificaciones.

$$\dot{r_{i}}=5(1+cos (r_{i})-\cos_{+}(y_{i})+\sigma_{R_i}(t)+10M_R(t))$$

$$\dot{y_{i}}=0.04( \ 1+cos (y_{i})+\tanh(2\cos_{+}(r_{i})) \ )$$

$$\dot{l_{i}}=10(0.1+cos (l_{i})-\cos_{+}(y_{i})+\sigma_{L_i}(t)+10M_L(t))$$

donde $r_i$ son las fases de los potenciales de axión producidos en las neouronas que conducen los estímulos del oído **derecho** a las neuronas centrales localizadas en el tálamo, cuyas fases están dadas por $y_i$; y $l_i$ son las fases de los potenciales de las neuronas conductoras del lado **izquierdo**. 
$\sigma_{R_i}$ y $\sigma_{L_i}$ son los estímulos externos que entran por los oídos derecho e izquierdo, respectivamente, y $M(t)$ es una función que acopla a las neuronas conductoras del lado derecho y $M_L(t)$ acopla a las del lado derecho.

In [1]:
using PyPlot
using TaylorSeries
println(VERSION)

using Compat;

INFO: Loading help data...


0.3.11


In [2]:
function step{T<:Number}(x::Taylor1{T},eps::T)
    orden=length(x.coeffs)
    hs=Number[]
    for i in 1:orden #esto lo hacemos para evitar problemas con los coeficientes de Taylor que 
        #son cero
        if x.coeffs[i]!=0
            push!(hs,x.coeffs[i])
        end 
    end 
    orden=length(hs)
    p1=1/orden
    p2=1/(orden-1)
    den1=hs[orden]
    den2=hs[orden-1]
    h1=(abs(eps/den1))^p1
    h2=(abs(eps/den2))^p2
    h=min(h1,h2)
    #println(hs)

    return h
end 
    

step (generic function with 1 method)

In [3]:
function d{T<:Real}(x::Taylor1{T}) # La función pide un Taylor
    res=(x+((x^2)^0.5))/2.0
    return res
#end    
#function positive{T<:Number}(x::Taylor1{T})
#    obj=(x+(x^2)^0.5)/2
 #   return obj
end 

d (generic function with 1 method)

In [4]:
function tanh(x::Taylor1)
     ap1 = exp(2.0*(x))
    res1 = (ap1-1.0)/(ap1+1.0)
    return res1
end 

tanh (generic function with 1 method)